# OPTIMIZATION BOILER PLATE

In [2]:
import torch
from torch.nn import functional as F

import torch.nn as nn
from torch import optim as optim

# wandb
import wandb
import pprint

# misc
import numpy as np
from os.path import join
import time

# visualizations
import plotly.graph_objects as go

# modules
from src.main_utils import Configuration
from src.evaluation_utils import eval_epoch
from src.thirdHand_data_loader import get_min_max_from_dataset
from src.motion_visualization_tools import quick_plot

ModuleNotFoundError: No module named 'src.evaluation_utils'

___


# WandB Settings

In [7]:
%env "WANDB_NOTEBOOK_NAME" "Optimization_boilerplate_11_VAECNN.ipynb"
wandb.login()

env: "WANDB_NOTEBOOK_NAME"="Optimization_boilerplate_11_VAECNN.ipynb"


AttributeError: module 'wandb' has no attribute 'login'

## Setting Sweep

In [8]:
sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

## Setting hyper-parameter options

In [9]:
parameters_dict = {
                'optimizer': {'values':['adam', 'rmsprop', 'sgd', 'nadam']}, # https://pytorch.org/docs/stable/optim.html
                'first_filter_size': {'values':[5, 6, 7, 8, 9, 10]},
                'latent_dim': {'values':[4, 5, 6, 7, 8]},
                'depth':{'values':[2, 3, 4]},
                'kernel_size': {'values':[3, 5]},
                'dropout': {'values':[0.1, 0.2, .4, .5]},
                'epochs': {'values':[300]}, 
                'learning_rate': {'values':[0.01, 0.001, 0.0001]},
                'batch_size': {'values':[128]},
                'reduction': {'values':['sum']}, #'mean'
                'kld_weight': {'values':[0.1, 1, 10]},
                'rec_loss': {'values':['L1', 'L2']},
            }

sweep_config['parameters'] = parameters_dict
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'values': [128]},
                'depth': {'values': [2, 3, 4]},
                'dropout': {'values': [0.1, 0.2, 0.4, 0.5]},
                'epochs': {'values': [300]},
                'first_filter_size': {'values': [5, 6, 7, 8, 9, 10]},
                'kernel_size': {'values': [3, 5]},
                'kld_weight': {'values': [0.1, 1, 10]},
                'latent_dim': {'values': [4, 5, 6, 7, 8]},
                'learning_rate': {'values': [0.01, 0.001, 0.0001]},
                'optimizer': {'values': ['adam', 'rmsprop', 'sgd', 'nadam']},
                'rec_loss': {'values': ['L1', 'L2']},
                'reduction': {'values': ['sum']}}}


In [10]:
sweep_id = wandb.sweep(sweep_config, project="thirdHand_VAECNN_Demo")

AttributeError: module 'wandb' has no attribute 'sweep'

---

# Model Basic works

In [2]:
model_config = Configuration()

Data loaded from 11 filse, stored in a dataframe with shape (248486, 10)
Dataframe headers are: ['px', 'py', 'pz', 'v1x', 'v1y', 'v1z', 'v2x', 'v2y', 'v2z', 'hand']
Data loaded from 2 filse, stored in a dataframe with shape (54402, 10)
Dataframe headers are: ['px', 'py', 'pz', 'v1x', 'v1y', 'v1z', 'v2x', 'v2y', 'v2z', 'hand']


### Dynamic model architecture

In [3]:
class Encoder(nn.Module):
    def __init__(self, device, first_filter_size, kernel_size, depth, dropout, latent_dim):
        super(Encoder, self).__init__()
        self.device= device
        self.first_filter_size= first_filter_size
        self.kernel_size= kernel_size
        self.encoder_padding = kernel_size//2 -1
        self.depth = depth 
        self.latent_dim = 2**latent_dim
        self.filter_number = [2**(i) for i in range(first_filter_size+1)]
        self.filter_number.reverse()
        
        self.filter_number = self.filter_number[:self.depth]
        self.last_encoder_filter_size = None
        
        self.dropout = dropout
        self.encoder_layers = self.make_encoder() 
        
        self.last_filter_size = self.filter_number[0]
        self.last_feature_size= (10-(depth*2+1))
        self.last_dim =  self.last_filter_size*self.last_feature_size

        self.flatten_layer = nn.Flatten().to(device)
        self.convert_to_latent = nn.Linear(self.last_dim, 2*self.latent_dim).to(device)
        
    def make_encoder(self):
        encoder_cnn_blocks = []
        
        for i in range(len(self.filter_number)):
            if i ==0:
                in_dim = 20
                out_dim = self.filter_number[i]   
            else:
                in_dim = self.filter_number[i-1]
                out_dim = self.filter_number[i]
                
            cnn_block_layers=[
                            nn.Conv1d(in_channels= in_dim, 
                                    out_channels= out_dim, 
                                    kernel_size= self.kernel_size, 
                                    padding= self.encoder_padding),
                            nn.BatchNorm1d(out_dim),
                            nn.ReLU(),
                            nn.Dropout(self.dropout),
                            ]
            
            cnn_block = nn.Sequential(*cnn_block_layers).to(self.device)
            
            encoder_cnn_blocks.append(cnn_block)
            self.last_encoder_filter_size = out_dim
            
        self.filter_number.reverse()
        
        return nn.ModuleList(encoder_cnn_blocks)
    
    
    def reparametrization(self, mean, log_var):
        """
        Samples from a normal distribution with a given set of
        means and log_vars
        """
        # epsilon is a vector of size (1, latent_dim)
        # it is samples from a Standard Normal distribution
        # mean = 0. and std = 1.
        epsilon = torch.normal(mean= 0, std= 1, size = log_var.shape).to(self.device) 

        # we need to convert log(var) into var:
        var = torch.exp(log_var*0.5)
        # epsilon = torch.randn_like(var)
        # now, we change the standard normal distributions to
        # a set of non standard normal distributions
        z = mean + epsilon*var
        return z
    
    def forward(self, x, y):
        
        for block in self.encoder_layers:
            x = block(x) 

        latent_ready = self.flatten_layer(x) 
        latent = self.convert_to_latent(latent_ready)

        mean = latent[:, : self.latent_dim]
        log_var = latent[:,self.latent_dim:]

        z = self.reparametrization(mean, log_var)

        return z, mean, log_var

In [4]:
# 3 or 5 both work!
kernel_size = 5

# between 10 and 5
first_filter_size = 9

# depth should be 2, 3, 4
depth = 2
dropout = 0.1
epochs = 300

batch_size = 128
latent_dim = 8

rec_loss= "L1"
reduction= "sum" 
kld_weight = 1e-1

last_filter_size = 256
last_feature_size = 5

model_ = Encoder(
                model_config.device, 
                first_filter_size, 
                kernel_size, 
                depth,
                dropout, 
                latent_dim
                )

sampler_size = 8
random_motion = torch.normal(mean= 0.1, std=0.35, size = (sampler_size, 20, 9)).to(model_config.device)
y = torch.ones(size=(sampler_size, 20, 1)).to(model_config.device)

print(random_motion.shape, y.shape)
z, mean, log_var = model_(random_motion, y)
print(z.shape)

torch.Size([8, 20, 9]) torch.Size([8, 20, 1])
torch.Size([8, 256])


In [5]:
class Decoder(nn.Module):
    def __init__(self, device, first_filter_size, kernel_size, depth, latent_dim, last_filter_size, last_feature_size):
        super(Decoder, self).__init__()
        
        self.device= device
        self.first_filter_size= first_filter_size
        self.kernel_size= kernel_size
        self.encoder_padding = kernel_size//2 -1
        self.depth = depth 
        self.latent_dim = 2**latent_dim
        self.last_filter_size= last_filter_size
        self.last_feature_size= last_feature_size
        
        
        
        self.filter_number = [2**(i) for i in range(first_filter_size+1)]        
        self.filter_number = self.filter_number[-self.depth:]
        self.filter_number.reverse()
        
        self.decoder_layers = self.make_decoder()
        self.z_to_decoder = nn.Linear(self.latent_dim+1,self.last_filter_size*self.last_feature_size).to(device)
       
    def make_decoder(self):
        decoder_cnn_blocks = []
        
        for i in range(len(self.filter_number)+1):
            self.decoder_padding = self.encoder_padding
            if i == 0:
                in_dim = self.last_filter_size
                out_dim = self.filter_number[i]
                
            elif i == len(self.filter_number):
                in_dim = self.filter_number[i-1]
                out_dim = 20  
                self.decoder_padding += 1
                
            else:
                in_dim = self.filter_number[i-1]
                out_dim = self.filter_number[i]
                        
            cnn_block_layers = [
                                nn.ConvTranspose1d(in_channels= in_dim, 
                                                    out_channels= out_dim, 
                                                    kernel_size= self.kernel_size, 
                                                    padding= self.decoder_padding,
                                                    ),
                                ]
            
            
            cnn_block = nn.Sequential(*cnn_block_layers).to(self.device)
            decoder_cnn_blocks.append(cnn_block)
            
        return nn.ModuleList(decoder_cnn_blocks)
    
    def forward(self, z, y):
        # print(z.shape, y.shape)
        fused_data =  torch.cat((z,y[:,0,:]), dim=1)
        # print( z.shape, y.shape, fused_data.shape)
        decoded = self.z_to_decoder(fused_data).view(-1, self.last_filter_size, self.last_feature_size)
        
        #decoded = self.z_to_decoder(z).view(-1, self.last_filter_size, self.last_feature_size)
        
        for block in self.decoder_layers:
            decoded = block(decoded)
        
        return decoded 

In [6]:
# 3 or 5 both work!
kernel_size = 5

# between 10 and 5
first_filter_size = 9

# depth should be 2, 3, 4
depth = 2
dropout = 0.1
epochs = 300

batch_size = 128
latent_dim = 8

rec_loss= "L1"
reduction= "sum" 
kld_weight = 1e-1

last_filter_size = 256
last_feature_size = 5

model_ = Decoder(model_config.device, 
                first_filter_size, 
                kernel_size, 
                depth, 
                latent_dim,
                last_filter_size,
                last_feature_size)

sampler_size = 8
random_z = torch.normal(mean= 0.1, std=0.35, size = (sampler_size, 2**latent_dim)).to(model_config.device)
y = torch.ones(size=(sampler_size, 20, 1)).to(model_config.device)
print(random_z.shape, y.shape)
x_generated = model_(random_z, y)
print(x_generated.shape)

torch.Size([8, 256]) torch.Size([8, 20, 1])
torch.Size([8, 20, 9])


Testing the encoder and decoder models

In [7]:
class VAE_CNN(nn.Module):
    def __init__(self, device, first_filter_size, kernel_size, depth, dropout, latent_dim, rec_loss, reduction, kld_weight):
        # check this out: https://stackoverflow.com/questions/59285058/batch-normalization-layer-for-cnn-lstm
        super(VAE_CNN, self).__init__()

        self.encoder = Encoder(
                                device, 
                                first_filter_size, 
                                kernel_size, 
                                depth, 
                                dropout, 
                                latent_dim,
                                )

        self.decoder = Decoder(
                                device, 
                                first_filter_size, 
                                kernel_size, 
                                depth, 
                                latent_dim, 
                                self.encoder.last_filter_size,
                                self.encoder.last_feature_size,
                                )
        
        self.reduction = reduction
        self.kld_weight = kld_weight
        self.rec_loss = rec_loss
        
    def vae_loss_function(self, x, x_rec, log_var, mean):
        if self.rec_loss == "L1":
            train_rec_loss = F.l1_loss(x_rec, x, reduction=self.reduction)   
        else:
            train_rec_loss = F.mse_loss(x_rec, x, reduction=self.reduction)     
        train_kld_loss = torch.mean(-0.5 * torch.sum(1 + log_var - mean**2 - log_var.exp(), dim = 1), dim = 0)

        train_loss = train_rec_loss  + train_kld_loss*self.kld_weight
        
        return train_loss, train_rec_loss, train_kld_loss*self.kld_weight  
    
    def forward(self, x, y):
        z, mean, log_var = self.encoder(x, y)
        x_rec = self.decoder(z, y)
        return x_rec, mean, log_var
        

In [8]:
def vae_loss_function(x, x_rec, log_var, mean, rec_loss, reduction, kld_weight):
    if rec_loss == "L1":
        train_rec_loss = F.l1_loss(x_rec, x, reduction=reduction)   
    else:
        train_rec_loss = F.mse_loss(x_rec, x, reduction=reduction)     
    train_kld_loss = torch.mean(-0.5 * torch.sum(1 + log_var - mean**2 - log_var.exp(), dim = 1), dim = 0)

    train_loss = train_rec_loss  + train_kld_loss*kld_weight
    
    return train_loss, train_rec_loss, train_kld_loss*kld_weight

## Training functions

---
### Only for training outside of **wandb**

In [9]:
def train_epoch_single(model, model_config, optimizer):
    model.train()
    epoch_loss = 0
    epoch_rec_loss = 0
    epoch_kld_loss = 0
 
    for data in model_config.train_iterator:
        x= data[model_config.data_item]
        y= data["Y"]
        
        optimizer.zero_grad()
        x_rec, mean, log_var = model(x,y)
        
        # calculating losses
        train_loss, train_rec_loss, train_kld_loss = vae_loss_function (x, 
                                                                        x_rec, 
                                                                        log_var, 
                                                                        mean, 
                                                                        rec_loss= model.rec_loss, 
                                                                        reduction= model.reduction,
                                                                        kld_weight= model.kld_weight)

        # updating the history
        epoch_rec_loss += train_rec_loss.item()
        epoch_kld_loss += train_kld_loss.item()
        epoch_loss += train_loss.item() 
        
        train_loss.backward()
        optimizer.step()
 
    counter = len(model_config.train_iterator)   
    results = [epoch_loss/counter,
                epoch_rec_loss/counter, 
                epoch_kld_loss/counter]      
    
    return results

In [10]:
def train_single(model, epochs, batch_size):  
    start_time = time.time()
    optimizer = optim.Adam(model.parameters(), lr = 0.0001)
    
    
    model_config.batch_size = batch_size
    model_config.process_dataset_dataloaders()
    train_losses = []
    train_rec_losses = []
    train_kld_losses = []
    eval_losses = []
    for epoch in range(epochs+1):
        # train
        train_loss, train_rec_loss, train_kld_loss = train_epoch_single(
                                                                        model, 
                                                                        model_config,
                                                                        optimizer,
                                                                        )
        log_plot= False
        report_interval = 50
        if epoch % report_interval ==0:
            log_plot= True
            path_to_save_plot = "runs/progress/tmp_fig_{}.png".format(epoch)  
            print("<<Image {} saved>>".format(epoch)) 
              
        eval_loss, eval_rec_losses, eval_kld_loss = eval_epoch(
                                                                model, 
                                                                model_config, 
                                                                model_config.valid_iterator, 
                                                                loss_function= vae_loss_function,
                                                                rec_loss= model.rec_loss, 
                                                                reduction=model.reduction, 
                                                                kld_weight= model.kld_weight,
                                                                save_plot= log_plot,
                                                                path_to_save_plot= path_to_save_plot,
                                                                is_vae= True
                                                                )
        
        train_losses.append(train_loss)
        train_rec_losses.append(train_rec_loss)
        train_kld_losses.append(train_kld_loss)
        eval_losses.append(eval_loss)
        
        if epoch % report_interval ==0:
            print("{}:\tTotal: {:.5f}\tEval loss: {:.5f}\t Rec loss: {:.5f}\t KLD loss: {:.5f}\t time: {:.1f}s".format(epoch, 
                                                                          train_loss,
                                                                          eval_loss,
                                                                          train_rec_loss,
                                                                          train_kld_loss,
                                                                          time.time()-start_time))
    quick_plot(train_losses, train_rec_losses, train_kld_losses, eval_losses)
    return train_losses ,train_rec_losses,train_kld_losses, eval_losses

In [11]:
def quick_plot(train_losses=None, rec= None, KLD= None, eval_losses= None, log_scale= True):
    items_to_plot= []
    if train_losses:
        train = go.Scatter(x= np.arange(len(train_losses)), y=train_losses, name="training", mode='lines')
        items_to_plot.append(train)
    if eval_losses:
        valid = go.Scatter(x= np.arange(len(eval_losses)),  y=eval_losses, name="validation", mode='lines')
        items_to_plot.append(valid)
    
    if rec:
        rec = go.Scatter(x= np.arange(len(rec)), y=rec, name="reconstruction", mode='lines')
        items_to_plot.append(rec)
    
    if KLD:
        KLD = go.Scatter(x= np.arange(len(KLD)), y=KLD, name="KLD", mode='lines')
        items_to_plot.append(KLD)

    fig = go.Figure(items_to_plot)
    
    if log_scale:
        fig.update_yaxes(type="log")
        
    fig.update_layout(template =  "plotly_dark")
    fig.show()

___

### Testingthe parameters and workflow manually

This model is based on the optimization results:

In [12]:
# 3 or 5 both work!
kernel_size = 5

# between 10 and 5
first_filter_size = 9

# depth should be 2, 3, 4
depth = 2
dropout = 0.1
epochs = 1000

batch_size = 128
latent_dim = 8

rec_loss= "L1"
reduction= "sum" 
kld_weight = 1e-1

model = VAE_CNN(model_config.device, 
                first_filter_size, 
                kernel_size, 
                depth, 
                dropout,
                latent_dim,
                rec_loss= rec_loss,
                reduction= reduction,
                kld_weight = kld_weight)

In [13]:
train_losses ,train_rec_losses,train_kld_losses, eval_losses = train_single (model, epochs, batch_size) 

<<Image 0 saved>>
0:	Total: 6051.03474	Eval loss: 7788.66113	 Rec loss: 6044.50319	 KLD loss: 6.53162	 time: 3.9s
<<Image 50 saved>>
50:	Total: 508.49157	Eval loss: 601.87650	 Rec loss: 451.68466	 KLD loss: 56.80691	 time: 10.8s
<<Image 100 saved>>
100:	Total: 378.09892	Eval loss: 296.09235	 Rec loss: 321.58685	 KLD loss: 56.51208	 time: 17.8s
<<Image 150 saved>>
150:	Total: 338.66573	Eval loss: 311.29185	 Rec loss: 283.43720	 KLD loss: 55.22853	 time: 24.6s
<<Image 200 saved>>
200:	Total: 313.98013	Eval loss: 272.37785	 Rec loss: 259.62895	 KLD loss: 54.35118	 time: 31.4s
<<Image 250 saved>>
250:	Total: 301.20442	Eval loss: 248.89674	 Rec loss: 247.57395	 KLD loss: 53.63048	 time: 38.2s
<<Image 300 saved>>
300:	Total: 287.24017	Eval loss: 236.58934	 Rec loss: 234.60218	 KLD loss: 52.63799	 time: 45.1s
<<Image 350 saved>>
350:	Total: 280.66426	Eval loss: 234.13605	 Rec loss: 228.35638	 KLD loss: 52.30788	 time: 51.9s
<<Image 400 saved>>
400:	Total: 274.82936	Eval loss: 234.42202	 Rec l

In [1]:
quick_plot(train_losses=train_losses, eval_losses=eval_losses, KLD=None, rec=None)

NameError: name 'quick_plot' is not defined

In [ ]:
quick_plot(train_losses=train_losses, eval_losses=eval_losses, KLD=train_kld_losses, rec= train_rec_losses)

In [35]:
def show_generated_motions(motion_data, index = 0):
    all_plots = []
    colors= ['white', 'red', 'blue']
    names = ["Original", "Rec", "Gen"]
    for i, stroke_date in enumerate(motion_data):
        sample = stroke_date[index,:,:] 
        x_rec_plot = go.Scatter3d(x= sample[:,0].detach().cpu().numpy(), 
                                    y= sample[: ,1].detach().cpu().numpy(), 
                                    z= sample[:, 2].detach().cpu().numpy(),  
                                    mode='lines',
                                    name=names[i],
                                    line=dict(width=3, color= colors[i]),
                                    )
        all_plots.append(x_rec_plot)


    fig = go.Figure(data=all_plots)
    plot_title = "Generated Motion"
    
    fig.update_layout(height=500,
                        margin=dict(l=5, r=5, t=50, b=50),
                        template =  "plotly_dark",
                        title_text= plot_title,
                        font=dict(family="Roboto, monospace",
                                size=12,
                                color="white"
                                ),
                        scene=dict(
                                    aspectratio = dict( x=1, y=1, z=1 ),
                                    camera=dict(up=dict(x=0, y=0, z=1),eye=dict(x=-1.5, y=1.5, z=1.5)),
                                ),
                        showlegend= True,
                        coloraxis_showscale=False,)

    fig.update_xaxes(showticklabels=False, 
                        showgrid=False, 
                        zeroline=False,
                        zerolinewidth=1, 
                        zerolinecolor='gray',
                        fixedrange= True,)

    fig.update_yaxes(showticklabels=False, 
                        showgrid=False, 
                        zeroline=False,
                        zerolinewidth=1, 
                        zerolinecolor='gray',
                        fixedrange= True,
                        scaleanchor = "x",
                        scaleratio = 1,)

    fig.show()

In [42]:
counter = 0
for d in model_config.train_iterator:
    if counter > 6:
        X = d["X"]
                
        x_samples = d[model_config.data_item]
        y_samples = d["Y"]
        
        x_rec,__, __ = model(x_samples, y_samples)
        
        z, __, __ = model.encoder(x_samples, y_samples)
        
        noise = torch.normal(mean=.1, std=.35, size = z.shape).to(model_config.device)
        z = z + noise
        x_gen = model.decoder(z , y_samples)
        
        

        centered_min_val= d["centered_min_val"][0]
        centered_max_val= d["centered_max_val"][0]
        center_points = torch.zeros_like(X[:, 0:1, :])        
        center_points[:, 0, :3] = X[:, 9, :3]

        x_rec_real_size = (x_rec*(centered_max_val - centered_min_val)+centered_min_val)#+center_points
        x_gen_real_size = (x_gen*(centered_max_val - centered_min_val)+centered_min_val)#+center_points    
        show_generated_motions([X, x_rec_real_size, x_gen_real_size], 3)
        
        print(x_gen_real_size.shape)
        np.save("some_test_motions_centered_gen", x_gen_real_size.detach().cpu().numpy())
        np.save("some_test_motions_centered_orig",  d["X_centered"].detach().cpu().numpy())
        break
    counter += 1

torch.Size([128, 20, 9])


In [24]:
sample_data= None
sample_size = np.random.randint(0, 128)

for d in model_config.train_iterator:
    x_samples = d[model_config.data_item][sample_size:sample_size+1, :,:]
    y_sampels = d['Y'][sample_size:sample_size+1, :,:] 

    z, __, __ = model.encoder(x_samples, y_sampels)

    noise = torch.normal(mean=.1, std=.1, size = z.shape).to(model_config.device)
    z = z + noise

    x_generated = model.decoder(z , y_sampels)
    x_rec,__, __ = model(x_samples, y_sampels)
    show_generated_motions([x_samples, x_rec,x_generated])
    break


In [25]:
sample_data= None
sample_size = np.random.randint(0, 128)
print(sample_size)

for d in model_config.train_iterator:
    x_samples = d[model_config.data_item][sample_size:sample_size+1, :,:]
    y_sampels = d['Y'][sample_size:sample_size+1, :,:] 
    
    min_val = d["centered_min_val"][sample_size:sample_size+1,:].view(1, 1, 9)
    max_val = d["centered_max_val"][sample_size:sample_size+1,:].view(1, 1, 9)
    
    print(min_val)
    # z, __, __ = model.encoder(x_samples, y_sampels)

    # noise = torch.normal(mean=.1, std=.1, size = z.shape).to(model_config.device)
    # z = z + noise

    # x_generated = model.decoder(z , y_sampels)
    # x_rec,__, __ = model(x_samples, y_sampels)
    
    
    x_samples = (x_samples*(max_val-min_val)) + min_val
    
    center_points = torch.zeros_like(d['X'][:, 0:1, :])
    center_points[:, 0, :3] = d['X'][:, 9, :3]       
    x_samples = x_samples+ center_points
    
    show_generated_motions([d["X"], x_samples])
    
    break


18
tensor([[[-0.0060, -0.0634, -0.0110,  0.1333, -0.3281, -0.9858, -0.9896,
          -0.9964, -1.0000]]], device='cuda:0')


In [26]:

for d in model_config.train_iterator:
    X = d["X"]
    min_val, max_val = get_min_max_from_dataset(X)
        
    X_scaled = torch.zeros_like(X)
    X_centered = torch.zeros_like(X)
    X_centered_scaled = torch.zeros_like(X)

    # # finding the touching point of each motion and centering the motion on that
    center_points = torch.zeros_like(X[:, 0:1, :])        
    center_points[:, 0, :3] = X[:, 9, :3]
    X_centered =  X - center_points

    # # scaling data between 0 and 1
    X_scaled =(X - min_val) / (max_val - min_val)
    
    print (torch.sum(X -(X_scaled*(max_val - min_val)+min_val)) < 0.001)
    
    # # scaling the centered data between 0 and 1
    centered_min_val, centered_max_val = get_min_max_from_dataset(X_centered)
    
    X_centered_scaled = (X_centered - centered_min_val) / (centered_max_val - centered_min_val)
    X_back_to_orig = (X_centered_scaled*(centered_max_val - centered_min_val)+centered_min_val)+center_points
    print (torch.sum(X -X_back_to_orig) < 0.001)
    break

tensor(True, device='cuda:0')
tensor(True, device='cuda:0')


In [27]:
counter = 0
for d in model_config.train_iterator:
    if counter > 6:
        X = d["X"]
        print(X.shape)
        min_val, max_val = get_min_max_from_dataset(X)
            
        X_scaled = torch.zeros_like(X)
        X_centered = torch.zeros_like(X)
        X_centered_scaled = torch.zeros_like(X)

        # # finding the touching point of each motion and centering the motion on that
        center_points = torch.zeros_like(X[:, 0:1, :])        
        center_points[:, 0, :3] = X[:, 9, :3]
        X_centered =  X - center_points

        # # scaling data between 0 and 1
        X_scaled =(X - min_val) / (max_val - min_val)
        
        print (torch.sum(X -(X_scaled*(max_val - min_val)+min_val)) < 0.001)
        
        # # scaling the centered data between 0 and 1
        centered_min_val, centered_max_val = get_min_max_from_dataset(X_centered)
        
        x_samples = d[model_config.data_item]
        y_samples = d["Y"]
        
        x_rec, __, __ = model(x_samples, y_samples)
        
        
        
        centered_min_val= d["centered_min_val"][0]
        centered_max_val= d["centered_max_val"][0]
        
        X_centered_scaled = (X_centered - centered_min_val) / (centered_max_val - centered_min_val)
        X_back_to_orig = (X_centered_scaled*(centered_max_val - centered_min_val)+centered_min_val)+center_points
            
        show_generated_motions([X, X_back_to_orig], 3)
        
        break
    counter += 1

torch.Size([128, 20, 9])
tensor(True, device='cuda:0')


___
# WandB Process

### Supporting functions

In [11]:
def optimizer_selector(model, optimizer_name, lr):
    if optimizer_name == "adam":
        optimzer = optim.Adam(model.parameters(), lr = lr)
        
    elif optimizer_name == "rmsprop":
        optimzer = optim.RMSprop(model.parameters(), lr = lr)
        
    elif optimizer_name == "sgd":
        optimzer = optim.SGD(model.parameters(), lr = lr)
        
    elif optimizer_name == "nadam":
        optimzer = optim.NAdam(model.parameters(), lr = lr)
        
    return optimzer

def dataset_maker(batch_size, model_config):
    model_config.batch_size = batch_size
    model_config.process_dataset_dataloaders() 
    
    
def train_epoch(model, model_config, optimizer):
    model.train()
    epoch_loss = 0
    epoch_rec_loss = 0
    epoch_kld_loss = 0
    
    for data in model_config.train_iterator:
        x= data[model_config.data_item]
        y= data["Y"]
        
        optimizer.zero_grad()
        x_rec, mean, log_var = model(x,y)
        
        
        train_loss, train_rec_loss, train_kld_loss = vae_loss_function (x, 
                                                                        x_rec, 
                                                                        log_var, 
                                                                        mean, 
                                                                        rec_loss= model.rec_loss, 
                                                                        reduction= model.reduction,
                                                                        kld_weight= model.kld_weight)
        
        # updating the history
        epoch_rec_loss += train_rec_loss.item()
        epoch_kld_loss += train_kld_loss.item()
        epoch_loss += train_loss.item() 
        
        train_loss.backward()
        optimizer.step()
    
        
                   
    counter = len(model_config.train_iterator)  
     
    results = [epoch_loss/counter,
                epoch_rec_loss/counter, 
                epoch_kld_loss/counter]   
  
    return results

---

# Optimization

In [12]:
def train(config=None):    

    path_to_save_plot = "runs/progress/tmp_fig.png"
    
    with wandb.init(config=config):
        config = wandb.config # this one basically iterates over the configuration settings

        # going through data, model, and optimizer variations
        dataset_maker(config.batch_size, model_config)

        model = VAE_CNN(device= model_config.device, 
                        first_filter_size= config.first_filter_size, 
                        kernel_size= config.kernel_size, 
                        depth= config.depth, 
                        dropout= config.dropout,
                        latent_dim= config.latent_dim,
                        rec_loss= config.rec_loss, 
                        reduction= config.reduction, 
                        kld_weight= config.kld_weight)

        optimizer= optimizer_selector(model, config.optimizer, config.learning_rate)
        
        # main train loop
        for epoch in range(config.epochs):
            train_loss, train_rec_loss, train_kld_loss = train_epoch(model, 
                                                                    model_config,
                                                                    optimizer)
            
            if epoch % 100 == 99:
                log_plot= True
            else:
                log_plot= False  

            try:
                eval_loss, eval_rec_losses, eval_kld_loss = eval_epoch(
                                                                model= model, 
                                                                config= model_config, 
                                                                data_iterator= model_config.valid_iterator, 
                                                                plot= False,
                                                                scaled_plot= False,
                                                                update_tensorboard= False,
                                                                epoch= epoch,
                                                                loss_function= vae_loss_function,
                                                                show_one_sample= False,
                                                                save_plot= log_plot,
                                                                path_to_save_plot = path_to_save_plot,
                                                                is_vae= True,
                                                                rec_loss= model.rec_loss, 
                                                                reduction=model.reduction, 
                                                                kld_weight= model.kld_weight,
                                                                )
            
                if epoch % 10 == 0:
                    # updating the training progress after each 10 epochs 
                    # we don't do it every epoch to save time
                    wandb.log(dict(loss= train_loss, epoch= epoch))
                    wandb.log(dict(loss_rec = train_rec_loss, epoch= epoch))
                    wandb.log(dict(loss_KLD= train_kld_loss, epoch= epoch))
                    wandb.log(dict(eval_loss=eval_loss,epoch= epoch))
                    wandb.log(dict(eval_rec=eval_rec_losses,epoch= epoch))
                    wandb.log(dict(eval_KLD=eval_kld_loss,epoch= epoch))
                
                if log_plot:
                    # saving an image of the process every 100 epochs
                    # if we do it more than this, it will kill time!
                    wandb.log({"eval_sample":wandb.Image(path_to_save_plot)})
            except:
                print ("something happend to the eval process, some issues with colors")

In [13]:
wandb.agent(sweep_id, train, count=200)

wandb: Agent Starting Run: ubfkfsfd with config:
wandb: 	batch_size: 128
wandb: 	depth: 4
wandb: 	dropout: 0.2
wandb: 	epochs: 300
wandb: 	first_filter_size: 10
wandb: 	kernel_size: 3
wandb: 	kld_weight: 1
wandb: 	latent_dim: 4
wandb: 	learning_rate: 0.01
wandb: 	optimizer: nadam
wandb: 	rec_loss: L1
wandb: 	reduction: sum
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
C:\Users\Ardavan\anaconda3\envs\thirdHand\lib\site-packages\IPython\html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
eval_KLD,▁▂▃▃▂▂▃▃▆▇▇▇▇▇▇▇█▇██▇█████▇▇▇▇
eval_loss,▃█▂▂▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval_rec,▃█▂▂▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_KLD,▁▃▃▃▃▃▃▃▆▇▇▇▇▇▇█▇█▇▇▇▇█▇▇▇▆▇▇▇
loss_rec,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,290
eval_KLD,20.59574
eval_loss,1761.31506
eval_rec,1740.71936


wandb: Agent Starting Run: ncqlear2 with config:
wandb: 	batch_size: 128
wandb: 	depth: 4
wandb: 	dropout: 0.2
wandb: 	epochs: 300
wandb: 	first_filter_size: 10
wandb: 	kernel_size: 5
wandb: 	kld_weight: 1
wandb: 	latent_dim: 8
wandb: 	learning_rate: 0.01
wandb: 	optimizer: adam
wandb: 	rec_loss: L2
wandb: 	reduction: sum
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
eval_KLD,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval_rec,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_KLD,▁██▇█▇████▇████▇▇█▇██▇▇▇▇███▇▇
loss_rec,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,290
eval_KLD,42.50547
eval_loss,45782.20898
eval_rec,45739.70312


wandb: Agent Starting Run: bui0dt86 with config:
wandb: 	batch_size: 128
wandb: 	depth: 3
wandb: 	dropout: 0.4
wandb: 	epochs: 300
wandb: 	first_filter_size: 6
wandb: 	kernel_size: 3
wandb: 	kld_weight: 10
wandb: 	latent_dim: 8
wandb: 	learning_rate: 0.01
wandb: 	optimizer: sgd
wandb: 	rec_loss: L1
wandb: 	reduction: sum
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

wandb: Agent Starting Run: q4bxw1tn with config:
wandb: 	batch_size: 128
wandb: 	depth: 4
wandb: 	dropout: 0.5
wandb: 	epochs: 300
wandb: 	first_filter_size: 9
wandb: 	kernel_size: 5
wandb: 	kld_weight: 10
wandb: 	latent_dim: 8
wandb: 	learning_rate: 0.01
wandb: 	optimizer: sgd
wandb: 	rec_loss: L1
wandb: 	reduction: sum
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


something happend to the eval process, some issues with colors
something happend to the eval process, some issues with colors
something happend to the eval process, some issues with colors


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
eval_KLD,
eval_loss,
eval_rec,
loss,
loss_KLD,
loss_rec,
epoch,290
eval_KLD,nan
eval_loss,nan
eval_rec,nan


wandb: Agent Starting Run: 1qamcwlx with config:
wandb: 	batch_size: 128
wandb: 	depth: 3
wandb: 	dropout: 0.4
wandb: 	epochs: 300
wandb: 	first_filter_size: 8
wandb: 	kernel_size: 3
wandb: 	kld_weight: 10
wandb: 	latent_dim: 8
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: adam
wandb: 	rec_loss: L1
wandb: 	reduction: sum
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
eval_KLD,▁█▇▆▆▆▆▆▆▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃
eval_loss,█▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval_rec,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_KLD,██▆▅▅▅▄▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
loss_rec,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,290
eval_KLD,108.76836
eval_loss,759.41064
eval_rec,650.64227


---

## Inserting the optimized hyper-parameters

Once the model is optimized, fill in the blanks and test the model

In [24]:
# fine-tuned parameter
batch_size = 128
model_depth = 9
latent_dim = 8
dropout = 0.5
loss_fn = "mse"
optimizer_name = "adam"
learning_rate = 0.001
epochs= 100

In [25]:
dataset_maker(batch_size, model_config)
model= model_maker(model_depth, latent_dim, dropout, model_config)
loss_function = loss_selector(loss_fn)
optimizer= optimizer_selector(model, optimizer_name, learning_rate)
dataloader = model_config.train_iterator

adam


AttributeError: 'NoneType' object has no attribute 'parameters'

In [11]:
eval_model(model, model_config, model_config.train_iterator, plot=True, scaled_plot= False, loss_function=loss_function)

0.0015138927847146988

In [10]:
train = go.Scatter(x= np.arange(len(model_train_loss)), y=model_train_loss, name="training")
valid = go.Scatter(x= np.arange(len(mdoel_eval_loss)),  y=mdoel_eval_loss, name="validation")

fig = go.Figure([train, valid])
fig.update_layout(template =  "plotly_dark")
fig.show()